In [ ]:
import pandas as pd
import re
import self_RAG
from tqdm.notebook import tqdm_notebook
import json
import pprint
import os
import shutil
df=pd.read_csv("../../data.csv")
df['University'] = df['University'].str.lower()

queries = [
    "What are this institute's rules or policies on maintaining academic integrity in the context of AI use, specifically regarding plagiarism and unauthorized assistance in academic work?",
    "What guidelines does this institute provide for instructors to effectively communicate expectations for AI use in coursework?",
    "What policies or guidelines does this institute have for addressing data privacy and security concerns when inputting sensitive or confidential information into AI systems?",
    "What ethical considerations does this institute have for addressing concerns such as bias, misinformation, and the perpetuation of inequalities in AI use?",
    "Does this institute commit to continuous adaptation or policy evaluation to meet educational needs amid the fast-evolving nature of AI? If so, how does it aim to regularly review and update its guidelines on AI usage?",
    "What requirements does this institute have for documenting the use of AI tools to ensure transparency on how AI contributes to academic work?",
    "What level of discretion does this institute grant instructors in establishing their own guidelines for AI usage in academic settings?"
]


df["University"]=df["University"].apply(lambda s: s.replace("\u2013"," "))
df["University"] = df["University"].apply(lambda s: ' '.join(s.replace('\xa0', ' ').title().split()))
df["University"] = df["University"].replace('Uci', 'University of California at Irvine')
df["University"] = df["University"].replace('Uc Berkeley', 'University of California at Berkeley')

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
#  regularly reviewing and updating their guidelines to meet educational needs in the fast-evolving nature of AI.

In [3]:
def get_institution_level_answers(df_institute, query):
    try:
        # Deduplicate URLs to avoid repeated calls to the same document
        urls = list(set(df_institute["Link"]))
        
        # Attempt to execute RAG query
        answer = self_RAG.execute_self_RAG(urls, query)
#         print("OUTPUT:")
#         pprint.pp(answer)
#         print("----"*5)
        return answer

    except Exception as e:
        print(f"Error: {e}")
        return f"No content could be found by the RAG when attempting to search for {query}."
    


def get_query_level_answers(df, query, query_answer=None):
    if query_answer is None:
        query_answer = {}

    list_of_institutes = list(set(df["University"]))
    
    # Iterate over each unique institution
    for institution in tqdm_notebook(list_of_institutes):
        if institution not in query_answer:
            df_institute = df[df["University"] == institution].reset_index(drop=True)
            # Create a unique query for each institution
            #mod_query = f"You are provided with the AI policies of {institution}. " + query.replace("institute", institution)
            
            # Get the answer and store it in query_answer for the institution
            query_answer[institution] = get_institution_level_answers(df_institute, query)
        else:
            print(f"Skipping {institution} as it already exists in query_answer.")
    
    return query_answer



def get_answers_to_queries(df, queries, query_answer_dict=None, save_path="query_answers.json"):
    if query_answer_dict is None:
        query_answer_dict = {}

    for query in tqdm_notebook(queries):
        # Initialize a dictionary for the query if it does not exist
        if query not in query_answer_dict:
            query_answer_dict[query] = {}

        # Ensure we answer each unique institution for the query
        if len(query_answer_dict[query]) != len(set(df["University"])):
            query_answer_dict[query] = get_query_level_answers(df, query, query_answer_dict[query])
            
            # Save the dictionary incrementally
            with open(save_path, 'w') as json_file:
                json.dump(query_answer_dict, json_file, indent=4)
        else:
            print(f"Skipping query '{query}' as it is already fully answered.")
    
    return query_answer_dict


In [4]:
query_answer_dict=get_answers_to_queries(df, queries)

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Deleted collection: langchain
6 DOCUMENTS LOADED, BEGINNING ANALYSIS...
Document count in vectorstore after adding new documents: 80
---RETRIEVE---


C:\Users\charl\Box\AI Policy Report 2024\AI and RAG model\2_summarizing_themes_at_an_individual_level\self_RAG.py:216: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  documents = retriever.get_relevant_documents(question)


"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
'\n---\n'
---GENERATE---


...

---CHECK HALLUCINATIONS---
Grade: YES
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
'\n---\n'
Deleted collection: langchain
9 DOCUMENTS LOADED, BEGINNING ANALYSIS...
Document count in vectorstore after adding new documents: 218
---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
'\n---\n'
---GENERATE---


C:\Users\charl\anaconda3\envs\pytorch_env\lib\site-packages\langsmith\client.py:311: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


---CHECK HALLUCINATIONS---
Grade: YES
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
'\n---\n'
Deleted collection: langchain
2 DOCUMENTS LOADED, BEGINNING ANALYSIS...
Document count in vectorstore after adding new documents: 43
---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
'\n---\n'
---GENERATE---


C:\Users\charl\anaconda3\envs\pytorch_env\lib\site-packages\langsmith\client.py:311: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


---CHECK HALLUCINATIONS---
Grade: YES
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
'\n---\n'
